In [1]:
X = [['Sunny','Warm','Normal','Strong','Warm','Same','+'],
 ['Sunny','Warm','High','Strong','Warm','Same','+'],
 ['Rainy','Cold','High','Strong','Warm','Change','-'],
 ['Sunny','Warm','High','Strong','Cool','Change','+']]

In [2]:
h = ['0','0','0','0','0','0']

In [3]:
def positiveInstances():
    return [x for x in X if x[-1]=='+']

In [8]:
for x in positiveInstances():
    for i in range(len(h)):
        if h[i]!=x[i]:
            if h[i]=='0':
                h[i]=x[i]
            else:
                h[i]='!'

In [9]:
h

['Sunny', 'Warm', '!', 'Strong', '!', '!']

Implement **List-Then-Eliminate** and **Candidate Elimination** below

In [1]:
# Candidate Elimination

class Holder:
    factors={}  #Un dictionar gol
    attributes = ()  #Parametrii sunt declarati

    def __init__(self,attr):  #
        self.attributes = attr
        for i in attr:
            self.factors[i]=[]
            
            
    def add_values(self,factor,values):
        self.factors[factor]=values

class CandidateElimination:
    Positive={} #Dictionarul pozitiv (gol)
    Negative={} #Dictionarul negativ (gol)
    
    def __init__(self,data,fact):
        self.num_factors = len(data[0][0])
        self.factors = fact.factors
        self.attr = fact.attributes
        self.dataset = data
        
        #print self.attr
        
    def run_algorithm(self):
#        print self.dataset

        # Limita specifica si generala

        G = self.initializeG()
        S = self.initializeS()
        

        # Popularea listei in variabila trial_set 

        count=0
        for trial_set in self.dataset:
            if self.is_positive(trial_set):
                G = self.remove_inconsistent_G(G,trial_set[0]) # Daca trial_set este alcatuit din exemple pozitive inlaturam datele inconsistente din limita generala
                S_new = S[:]
                print S_new
                for s in S:
                    if not self.consistent(s,trial_set[0]):
                        S_new.remove(s)
                        generalization = self.generalize_inconsistent_S(s,trial_set[0])
                        generalization = self.get_general(generalization,G)
                        if generalization:
                            S_new.append(generalization)
                    S = S_new[:]
                    S = self.remove_more_general(S)
                   
                    print S
            else:
                S = self.remove_inconsistent_S(S,trial_set[0]) #Daca este negativ inlaturam datele inconsistente din limita specifica
                G_new = G[:]
                print G_new
                for g in G:
                    if self.consistent(g,trial_set[0]):
                        G_new.remove(g)
                        specializations = self.specialize_inconsistent_G(g,trial_set[0])
                        specializationss = self.get_specific(specializations,S)
                        if specializations != []:
                            G_new += specializations
                    G = G_new[:] 
                    print G
                    G = self.remove_more_specific(G)
        
        print S
        print G
    
    def initializeS(self):
        # Limita specifica
        S = tuple(['-' for factor in range(self.num_factors)])
        return [S]

    def initializeG(self):
        # Limita generala
        G = tuple(['?' for factor in range(self.num_factors)])
        return [G]

    def is_positive(self,trial_set):
        # Verifica daca un trial_set dat este pozitiv
        if trial_set[1] == 'Y':
            return True
        elif trial_set[1] == 'N':
            return False
        else:
            raise TypeError("invalid target value")

    def is_negative(self,trial_set):
        # Verifica daca un trial_set dat este negativ
        if trial_set[1] == 'N':
            return False
        elif trial_set[1] == 'Y':
            return True
        else:
            raise TypeError("invalid target value")

    def match_factor(self,value1,value2):
        # Check for the factors values match, necessary while checking the consistency of training trial_set with the hypothesis
        if value1 == '?' or value2 == '?':
            return True
        elif value1 == value2 :
            return True
        return False

    def consistent(self,hypothesis,instance):
        # Verifica daca instanta este parte din ipoteza 
        for i,factor in enumerate(hypothesis):
            if not self.match_factor(factor,instance[i]):
                return False
        return True

    def remove_inconsistent_G(self,hypotheses,instance):
        # Pentru un trial_set pozitiv ipoteza inconsistenta din G trebuie inlaturata
        G_new = hypotheses[:]
        for g in hypotheses:
            if not self.consistent(g,instance):
                G_new.remove(g)
        return G_new

    def remove_inconsistent_S(self,hypotheses,instance):
        # Pentru un trial_set negativ ipoteza inconsistenta din S trebuie inlaturata
        S_new = hypotheses[:]
        for s in hypotheses:
            if self.consistent(s,instance):
                S_new.remove(s)
        return S_new
        
    def remove_more_general(self,hypotheses):
        # Dupa generalizarea S pentru un trial_set pozitiv, ipoteza S mai generala decat altele in S este inlaturata
        S_new = hypotheses[:]
        for old in hypotheses:
            for new in S_new:
                if old!=new and self.more_general(new,old):
                    S_new.remove[new]
        return S_new

    def remove_more_specific(self,hypotheses):
        # Dupa specializarea G pentru un trial_set negativ, ipoteza G mai specifica decat altele in G este inlaturata
        G_new = hypotheses[:]
        for old in hypotheses:
            for new in G_new:
                if old!=new and self.more_specific(new,old):
                    G_new.remove[new]
        return G_new

    def generalize_inconsistent_S(self,hypothesis,instance):
        # Atunci cand se gaseste o ipoteza inconsistenta pentru trial_set-ul pozitiv in limita specifica trebuie generalizata pentru a fi consistenta cu trial_set-ul
        hypo = list(hypothesis)
        for i,factor in enumerate(hypo):
            if factor == '-':
                hypo[i] = instance[i]
            elif not self.match_factor(factor,instance[i]):
                hypo[i] = '?'
        generalization = tuple(hypo)
        return generalization

    def specialize_inconsistent_G(self,hypothesis,instance):
        # Atunci cand se gaseste o ipoteza inconsistenta pentru trial_set-ul negativ in limita generala trebuie specializata pentru a fi consistenta cu trial_set-ul
        specializations = []
        hypo = list(hypothesis)
        for i,factor in enumerate(hypo):
            if factor == '?':
                values = self.factors[self.attr[i]]
                for j in values:
                    if instance[i] != j:
                        hyp=hypo[:]
                        hyp[i]=j
                        hyp=tuple(hyp)
                        specializations.append(hyp)
        return specializations

   
    def get_general(self,generalization,G): 
        for g in G:
            if self.more_general(g,generalization):
                return generalization
        return None

    def get_specific(self,specializations,S):
        valid_specializations = []
        for hypo in specializations:
            for s in S:
                if self.more_specific(s,hypo) or s==self.initializeS()[0]:
                    valid_specializations.append(hypo)
        return valid_specializations

   
    def get_version_space(self,specific,general):
        # Avand limita generala si specifica, evaluam VS dintre acestea
        while get_order(VS):  
            for hypothesis in VS[:]:
                hypo = list(hypothesis)
                for i,factor in enumerate(hypo):
                    if factor != '?':
                        hyp=hypo[:]
                        hyp[i]='?'
                        if self.exists_general(hyp,general)and self.exists_specific(hyp,specific):
                            VS.append(tuple(hyp))

        return VS

    def get_order(self,hypothesis):
        pass
    
    def more_general(self,hyp1,hyp2):
        # Verifica daca hyp1 este mai generala decat hyp2
        hyp = zip(hyp1,hyp2)
        for i,j in hyp:
            if i == '?':
                continue
            elif j == '?':
                if i != '?':
                    return False
            elif i != j:
                return False
            else:                       
                continue
        return True

    def more_specific(self,hyp1,hyp2):
        # Daca hyp1 este mai specific decat hyp2 este echivalentul la hyp2 mai generala decat hyp1
        return self.more_general(hyp2,hyp1)



dataset=[(('sunny','warm','normal','strong','warm','same'),'Y'),(('sunny','warm','high','strong','warm','same'),'Y'),(('rainy','cold','high','strong','warm','change'),'N'),(('sunny','warm','high','strong','cool','change'),'Y')]
attributes =('Sky','Temp','Humidity','Wind','Water','Forecast')


f = Holder(attributes)
f.add_values('Sky',('sunny','rainy','cloudy'))
f.add_values('Temp',('cold','warm'))
f.add_values('Humidity',('normal','high'))
f.add_values('Wind',('weak','strong'))
f.add_values('Water',('warm','cold'))
f.add_values('Forecast',('same','change'))
a = CandidateElimination(dataset,f)
a.run_algorithm()

[('-', '-', '-', '-', '-', '-')]
[('sunny', 'warm', 'normal', 'strong', 'warm', 'same')]
[('sunny', 'warm', 'normal', 'strong', 'warm', 'same')]
[('sunny', 'warm', '?', 'strong', 'warm', 'same')]
[('?', '?', '?', '?', '?', '?')]
[('sunny', '?', '?', '?', '?', '?'), ('cloudy', '?', '?', '?', '?', '?'), ('?', 'warm', '?', '?', '?', '?'), ('?', '?', 'normal', '?', '?', '?'), ('?', '?', '?', 'weak', '?', '?'), ('?', '?', '?', '?', 'cold', '?'), ('?', '?', '?', '?', '?', 'same')]
[('sunny', 'warm', '?', 'strong', 'warm', 'same')]
[('sunny', 'warm', '?', 'strong', '?', '?')]
[('sunny', 'warm', '?', 'strong', '?', '?')]
[('sunny', '?', '?', '?', '?', '?'), ('?', 'warm', '?', '?', '?', '?')]
